In [1]:
# ============================================================================
# 语音情绪识别 - Conformer V3.1 (微调冲刺版)
# 数据集: CREMA-D
# 修改: 
# 1. 自动根据当前文件名创建输出目录
# 2. V3.1 配置: 降低 Smoothing/Noise/Dropout 以提升上限
# ============================================================================

# =====================
# 0) 导入必要的库
# =====================
import os
import sys
import math
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix, f1_score

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

# 设置随机种子
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 设置绘图风格
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")


# =====================
# 1) 自动创建输出目录 (根据文件名)
# =====================
def create_experiment_dir(base_dir="VED/Output"):
    """
    创建带时间戳的实验目录，目录名基于当前脚本文件名
    """
    # 获取当前脚本的文件名（不带后缀）
    try:
        # 如果是 .py 脚本运行
        script_name = os.path.splitext(os.path.basename(__file__))[0]
    except NameError:
        # 如果是在 Jupyter Notebook 中运行，使用默认名称
        script_name = "conformer_v3_1_notebook"

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    exp_dir = os.path.join(base_dir, f"{script_name}_{timestamp}")
    
    # 确保基础路径存在
    os.makedirs(base_dir, exist_ok=True)
    os.makedirs(exp_dir, exist_ok=True)
    
    # 创建子目录
    subdirs = ["models", "plots", "logs"]
    for subdir in subdirs:
        os.makedirs(os.path.join(exp_dir, subdir), exist_ok=True)
    
    print(f"✓ Experiment directory created: {exp_dir}")
    return exp_dir


# 创建实验目录
EXP_DIR = create_experiment_dir()
MODEL_DIR = os.path.join(EXP_DIR, "models")
PLOT_DIR = os.path.join(EXP_DIR, "plots")
LOG_DIR = os.path.join(EXP_DIR, "logs")


# =====================
# 2) 数据路径配置
# =====================
AUDIO_DIR = Path("../AudioWAV")  # 请确保此路径正确
assert AUDIO_DIR.exists(), f"Audio directory not found: {AUDIO_DIR}"


# =====================
# 3) 构建元数据
# =====================
def build_metadata(audio_dir):
    """从文件名解析元数据"""
    records = []
    for wav_file in audio_dir.glob("*.wav"):
        filename = wav_file.stem
        parts = filename.split("_")
        if len(parts) == 4:
            speaker, sentence, emotion, intensity = parts
            records.append({
                "path": wav_file,
                "speaker": speaker,
                "sentence": sentence,
                "emotion": emotion,
                "intensity": intensity
            })
    return pd.DataFrame(records)


meta = build_metadata(AUDIO_DIR)
print(f"Total samples: {len(meta)}")
print(f"Emotion distribution:\n{meta['emotion'].value_counts()}")


# =====================
# 4) 音频特征提取配置
# =====================
SR = 16000
N_MELS = 64
FFT = 1024
HOP = 160
WIN = 400
FIXED_SECONDS = 3.0
MAX_FRAMES = int(math.ceil(FIXED_SECONDS * SR / HOP))

print(f"Feature shape: ({MAX_FRAMES}, {N_MELS}, 1)")


# =====================
# 5) Log-Mel 特征提取
# =====================
def load_logmel(path: Path):
    """加载音频并提取Log-Mel频谱图"""
    y, _ = librosa.load(path, sr=SR, mono=True)
    
    target_len = int(FIXED_SECONDS * SR)
    if len(y) < target_len:
        y = np.pad(y, (0, target_len - len(y)))
    else:
        y = y[:target_len]
    
    S = librosa.feature.melspectrogram(
        y=y, sr=SR, n_fft=FFT, hop_length=HOP,
        win_length=WIN, n_mels=N_MELS, power=2.0
    )
    S_db = librosa.power_to_db(S, ref=np.max).astype(np.float32)
    feat = np.transpose(S_db[..., None], (1, 0, 2))
    
    if feat.shape[0] < MAX_FRAMES:
        feat = np.pad(feat, ((0, MAX_FRAMES - feat.shape[0]), (0, 0), (0, 0)))
    else:
        feat = feat[:MAX_FRAMES]
    
    return feat


# =====================
# 6) 批量提取特征
# =====================
print("Extracting features...")
specs = []
emotions = []
speakers = []

for idx, row in meta.iterrows():
    if idx % 500 == 0:
        print(f"  Processing {idx}/{len(meta)}...")
    specs.append(load_logmel(row['path']))
    emotions.append(row['emotion'])
    speakers.append(row['speaker'])

X = np.stack(specs)
print(f"Feature matrix shape: {X.shape}")


# =====================
# 7) 标签编码 & 数据划分
# =====================
le_emo = LabelEncoder()
y = le_emo.fit_transform(emotions).astype(np.int32)
groups = np.array(speakers)

print(f"Classes: {le_emo.classes_}")
print(f"Number of classes: {len(le_emo.classes_)}")

# 说话人独立划分
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

Xtr, Xte = X[train_idx], X[test_idx]
ytr, yte = y[train_idx], y[test_idx]

print(f"Train: {Xtr.shape}, Test: {Xte.shape}")


# =====================
# 8) Z-Score 标准化
# =====================
mean_spec = Xtr.mean(axis=(0, 1, 2), keepdims=True)
std_spec = Xtr.std(axis=(0, 1, 2), keepdims=True) + 1e-6

Xtr = (Xtr - mean_spec) / std_spec
Xte = (Xte - mean_spec) / std_spec

print(f"Normalized - Train mean: {Xtr.mean():.4f}, std: {Xtr.std():.4f}")

# 保存标准化参数
np.save(os.path.join(MODEL_DIR, "norm_mean.npy"), mean_spec)
np.save(os.path.join(MODEL_DIR, "norm_std.npy"), std_spec)


# =====================
# 9) 自定义层和函数
# =====================

class SpecAugment(layers.Layer):
    """SpecAugment 数据增强层"""
    def __init__(self, time_mask_num=2, freq_mask_num=2, 
                 time_mask_max=16, freq_mask_max=6, **kwargs):
        super().__init__(**kwargs)
        self.time_mask_num = time_mask_num
        self.freq_mask_num = freq_mask_num
        self.time_mask_max = time_mask_max
        self.freq_mask_max = freq_mask_max
    
    def call(self, x, training=False):
        if not training:
            return x
        
        batch_size = tf.shape(x)[0]
        T = tf.shape(x)[1]
        F = tf.shape(x)[2]
        
        for _ in range(self.time_mask_num):
            t = tf.random.uniform([], 0, self.time_mask_max, dtype=tf.int32)
            t0 = tf.random.uniform([], 0, tf.maximum(1, T - t), dtype=tf.int32)
            mask = tf.concat([
                tf.ones([batch_size, t0, F, 1]),
                tf.zeros([batch_size, t, F, 1]),
                tf.ones([batch_size, T - t0 - t, F, 1])
            ], axis=1)
            x = x * mask
        
        for _ in range(self.freq_mask_num):
            f = tf.random.uniform([], 0, self.freq_mask_max, dtype=tf.int32)
            f0 = tf.random.uniform([], 0, tf.maximum(1, F - f), dtype=tf.int32)
            mask = tf.concat([
                tf.ones([batch_size, T, f0, 1]),
                tf.zeros([batch_size, T, f, 1]),
                tf.ones([batch_size, T, F - f0 - f, 1])
            ], axis=2)
            x = x * mask
        
        return x
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "time_mask_num": self.time_mask_num,
            "freq_mask_num": self.freq_mask_num,
            "time_mask_max": self.time_mask_max,
            "freq_mask_max": self.freq_mask_max,
        })
        return config


class LabelSmoothingFocalLoss(tf.keras.losses.Loss):
    """Label Smoothing + Focal Loss"""
    def __init__(self, smoothing=0.05, gamma=2.0, **kwargs):
        # V3.1 修改：默认 smoothing 降为 0.05
        super().__init__(**kwargs)
        self.smoothing = smoothing
        self.gamma = gamma
    
    def call(self, y_true, y_pred):
        n_classes = tf.cast(tf.shape(y_pred)[-1], tf.float32)
        y_true = tf.cast(y_true, tf.float32)
        
        # Label smoothing
        y_true = y_true * (1 - self.smoothing) + self.smoothing / n_classes
        
        # Focal loss
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        ce = -y_true * tf.math.log(y_pred)
        weight = tf.pow(1 - y_pred, self.gamma)
        focal = weight * ce
        
        return tf.reduce_sum(focal, axis=-1)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "smoothing": self.smoothing,
            "gamma": self.gamma,
        })
        return config


def glu_activation(x):
    """GLU激活函数"""
    channels = x.shape[-1] // 2
    return x[..., :channels] * tf.sigmoid(x[..., channels:])


def se_block(x, ratio=8):
    """SE通道注意力"""
    channels = x.shape[-1]
    squeeze = layers.GlobalAveragePooling2D()(x)
    excite = layers.Dense(max(channels // ratio, 4), activation="relu")(squeeze)
    excite = layers.Dense(channels, activation="sigmoid")(excite)
    excite = layers.Reshape((1, 1, channels))(excite)
    return layers.Multiply()([x, excite])


def attentive_pooling(x):
    """注意力池化"""
    attention = layers.Dense(1, activation="tanh")(x)
    attention = layers.Softmax(axis=1)(attention)
    pooled = tf.reduce_sum(x * attention, axis=1)
    return pooled


def multi_scale_conv_block(x):
    """多尺度卷积块"""
    # L2 正则保持 5e-5 (V3)
    conv3 = layers.Conv2D(32, (3, 3), padding="same", activation="relu",
                          kernel_regularizer=tf.keras.regularizers.l2(5e-5))(x)
    conv5 = layers.Conv2D(32, (5, 5), padding="same", activation="relu",
                          kernel_regularizer=tf.keras.regularizers.l2(5e-5))(x)
    conv7 = layers.Conv2D(32, (7, 7), padding="same", activation="relu",
                          kernel_regularizer=tf.keras.regularizers.l2(5e-5))(x)
    
    concat = layers.Concatenate()([conv3, conv5, conv7])
    concat = layers.BatchNormalization()(concat)
    concat = se_block(concat, ratio=8)
    concat = layers.MaxPool2D((2, 2))(concat)
    
    return concat


def conformer_block(x, d_model=128, num_heads=4, conv_kernel=15, dropout=0.15):
    """Conformer Block"""
    # Feed Forward Module 1
    ff1 = layers.Dense(d_model * 4, activation="swish")(x)
    ff1 = layers.Dropout(dropout)(ff1)
    ff1 = layers.Dense(d_model)(ff1)
    ff1 = layers.Dropout(dropout)(ff1)
    x = layers.LayerNormalization()(x + 0.5 * ff1)
    
    # Multi-Head Self-Attention
    attn = layers.MultiHeadAttention(
        num_heads=num_heads, 
        key_dim=d_model // num_heads,
        dropout=dropout
    )(x, x)
    x = layers.LayerNormalization()(x + attn)
    
    # Convolution Module
    conv = layers.Conv1D(d_model * 2, kernel_size=1)(x)
    conv = glu_activation(conv)
    conv = layers.DepthwiseConv1D(
        kernel_size=conv_kernel, 
        padding="same",
        depthwise_regularizer=tf.keras.regularizers.l2(5e-5)
    )(conv)
    conv = layers.BatchNormalization()(conv)
    conv = layers.Activation("swish")(conv)
    conv = layers.Conv1D(d_model, kernel_size=1)(conv)
    conv = layers.Dropout(dropout)(conv)
    x = layers.LayerNormalization()(x + conv)
    
    # Feed Forward Module 2
    ff2 = layers.Dense(d_model * 4, activation="swish")(x)
    ff2 = layers.Dropout(dropout)(ff2)
    ff2 = layers.Dense(d_model)(ff2)
    ff2 = layers.Dropout(dropout)(ff2)
    x = layers.LayerNormalization()(x + 0.5 * ff2)
    
    return x


# =====================
# 10) 构建 Conformer V3.1 模型
# =====================
def build_conformer_v3_1(input_shape, n_classes=6, d_model=128):
    """
    Conformer V3.1 - 微调冲刺版
    """
    inp = layers.Input(shape=input_shape)
    
    # V3.1 修改：噪声降低 0.03 -> 0.02
    x = layers.GaussianNoise(0.02)(inp)
    x = SpecAugment(time_mask_num=2, freq_mask_num=2, 
                    time_mask_max=16, freq_mask_max=6)(x)
    
    # 多尺度卷积前端
    x = multi_scale_conv_block(x)
    
    # 卷积层 1 - V3.1 修改：Dropout 0.25 -> 0.2
    x = layers.Conv2D(128, (3, 3), padding="same", activation="relu",
                      kernel_regularizer=tf.keras.regularizers.l2(5e-5))(x)
    x = layers.BatchNormalization()(x)
    x = se_block(x, ratio=8)
    x = layers.MaxPool2D((2, 2))(x)
    x = layers.SpatialDropout2D(0.2)(x)
    
    # 卷积层 2 - V3.1 修改：Dropout 0.25 -> 0.2
    x = layers.Conv2D(256, (3, 3), padding="same", activation="relu",
                      kernel_regularizer=tf.keras.regularizers.l2(5e-5))(x)
    x = layers.BatchNormalization()(x)
    x = se_block(x, ratio=8)
    x = layers.MaxPool2D((2, 2))(x)
    x = layers.SpatialDropout2D(0.2)(x)
    
    # Reshape for Conformer
    T_prime = x.shape[1]
    x = layers.Reshape((T_prime, -1))(x)
    x = layers.Dense(d_model)(x)
    x = layers.Dropout(0.15)(x)
    
    # Conformer Blocks
    x = conformer_block(x, d_model=d_model, num_heads=4, conv_kernel=15, dropout=0.15)
    x = conformer_block(x, d_model=d_model, num_heads=4, conv_kernel=15, dropout=0.15)
    
    # 注意力池化
    x = attentive_pooling(x)
    
    # 分类头 - V3.1 修改：Dropout 0.4 -> 0.3 / 0.3 -> 0.2
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation="relu", 
                     kernel_regularizer=tf.keras.regularizers.l2(5e-5))(x)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(n_classes, activation="softmax", name="emotion")(x)
    
    model = models.Model(inp, output)
    return model


# =====================
# 11) Mixup 数据生成器
# =====================
class MixupGenerator(tf.keras.utils.Sequence):
    """带Mixup的数据生成器"""
    def __init__(self, x, y, batch_size=64, alpha=0.2, shuffle=True):
        self.x = x
        self.y = y
        self.batch_size = batch_size
        self.alpha = alpha
        self.shuffle = shuffle
        self.indexes = np.arange(len(x))
        if shuffle:
            np.random.shuffle(self.indexes)
    
    def __len__(self):
        return int(np.ceil(len(self.x) / self.batch_size))
    
    def __getitem__(self, idx):
        batch_indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = self.x[batch_indexes].copy()
        batch_y = self.y[batch_indexes].copy()
        
        # 50%概率应用Mixup
        if np.random.random() < 0.5 and self.alpha > 0:
            lam = np.random.beta(self.alpha, self.alpha)
            shuffle_idx = np.random.permutation(len(batch_x))
            batch_x = lam * batch_x + (1 - lam) * batch_x[shuffle_idx]
            batch_y = lam * batch_y + (1 - lam) * batch_y[shuffle_idx]
        
        return batch_x, batch_y
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)


# =====================
# 12) 自定义回调
# =====================
class MacroF1Callback(callbacks.Callback):
    """计算并记录 Macro F1"""
    def __init__(self, X_val, y_val):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.best_f1 = 0
        self.f1_history = []
    
    def on_epoch_end(self, epoch, logs=None):
        y_pred = np.argmax(self.model.predict(self.X_val, verbose=0), axis=1)
        f1 = f1_score(self.y_val, y_pred, average="macro")
        self.f1_history.append(f1)
        logs = logs or {}
        logs["val_macro_f1"] = f1
        
        if f1 > self.best_f1:
            self.best_f1 = f1
            print(f" — val_macro_f1: {f1:.4f} ★ New Best!")
        else:
            print(f" — val_macro_f1: {f1:.4f}")


# =====================
# 13) 训练函数
# =====================
def train_conformer_v3_1(Xtr, ytr, Xte, yte, le_emo, 
                         epochs=100, batch_size=64, 
                         exp_dir=EXP_DIR, model_dir=MODEL_DIR, log_dir=LOG_DIR):
    """V3.1 完整训练流程"""
    
    tf.keras.backend.clear_session()
    
    n_classes = len(le_emo.classes_)
    model = build_conformer_v3_1(input_shape=Xtr.shape[1:], n_classes=n_classes)
    
    # 保存模型结构
    model.summary()
    with open(os.path.join(log_dir, "model_summary.txt"), "w") as f:
        model.summary(print_fn=lambda x: f.write(x + "\n"))
    
    # One-hot编码
    ytr_onehot = tf.keras.utils.to_categorical(ytr, n_classes)
    yte_onehot = tf.keras.utils.to_categorical(yte, n_classes)
    
    # 编译
    model.compile(
        optimizer=tf.keras.optimizers.AdamW(
            learning_rate=8e-4,
            weight_decay=2e-4
        ),
        # V3.1 修改：Smoothing 0.1 -> 0.05
        loss=LabelSmoothingFocalLoss(smoothing=0.05, gamma=2.0),
        metrics=["accuracy"]
    )
    
    # 学习率调度
    def lr_schedule(epoch):
        warmup_epochs = 5
        initial_lr = 8e-4
        
        if epoch < warmup_epochs:
            return initial_lr * (epoch + 1) / warmup_epochs
        else:
            progress = (epoch - warmup_epochs) / (epochs - warmup_epochs)
            return initial_lr * 0.5 * (1 + np.cos(np.pi * progress))
    
    # Mixup数据生成器
    train_gen = MixupGenerator(Xtr, ytr_onehot, batch_size=batch_size, alpha=0.2)
    
    # F1回调
    f1_callback = MacroF1Callback(Xte, yte)
    
    # 回调列表
    callbacks_list = [
        f1_callback,
        callbacks.ModelCheckpoint(
            os.path.join(model_dir, "best_model.h5"),
            monitor="val_macro_f1",
            mode="max",
            save_best_only=True,
            verbose=1
        ),
        callbacks.ModelCheckpoint(
            os.path.join(model_dir, "last_model.h5"),
            save_best_only=False,
            verbose=0
        ),
        callbacks.EarlyStopping(
            monitor="val_macro_f1",
            mode="max",
            patience=25,
            restore_best_weights=True,
            verbose=1
        ),
        callbacks.LearningRateScheduler(lr_schedule, verbose=0),
        callbacks.CSVLogger(os.path.join(log_dir, "training_log.csv"))
    ]
    
    # 保存训练配置
    config = {
        "model": "Conformer V3.1",
        "description": "Lower smoothing/noise/dropout compared to V3",
        "epochs": epochs,
        "batch_size": batch_size,
        "initial_lr": 8e-4,
        "weight_decay": 2e-4,
        "mixup_alpha": 0.2,
        "label_smoothing": 0.05,
        "focal_gamma": 2.0,
        "dropout_rates": "0.15/0.2/0.3/0.2",
        "l2_reg": 5e-5,
        "n_classes": n_classes
    }
    
    with open(os.path.join(log_dir, "config.txt"), "w") as f:
        for k, v in config.items():
            f.write(f"{k}: {v}\n")
    
    # 训练
    print("\n" + "=" * 60)
    print("Starting Conformer V3.1 training...")
    print(f"Output directory: {exp_dir}")
    print("=" * 60)
    
    history = model.fit(
        train_gen,
        validation_data=(Xte, yte_onehot),
        epochs=epochs,
        callbacks=callbacks_list,
        verbose=1
    )
    
    # 保存F1历史
    history.history['val_macro_f1'] = f1_callback.f1_history
    
    return model, history, f1_callback.best_f1


# =====================
# 14) 评估函数
# =====================
def evaluate_model(model, Xte, yte, le_emo, plot_dir=PLOT_DIR, log_dir=LOG_DIR):
    """完整模型评估"""
    
    # 预测
    y_pred_proba = model.predict(Xte, verbose=0)
    y_pred = np.argmax(y_pred_proba, axis=1)
    
    # 计算指标
    macro_f1 = f1_score(yte, y_pred, average="macro")
    weighted_f1 = f1_score(yte, y_pred, average="weighted")
    accuracy = np.mean(y_pred == yte)
    class_f1 = f1_score(yte, y_pred, average=None)
    
    # 分类报告
    report = classification_report(yte, y_pred, target_names=le_emo.classes_, digits=4)
    
    print("\n" + "=" * 60)
    print("Classification Report - Conformer V3.1")
    print("=" * 60)
    print(report)
    
    # 保存报告
    with open(os.path.join(log_dir, "classification_report.txt"), "w") as f:
        f.write("Classification Report - Conformer V3.1\n")
        f.write("=" * 60 + "\n")
        f.write(report)
        f.write(f"\nSummary:\n")
        f.write(f"  Accuracy:    {accuracy:.4f}\n")
        f.write(f"  Macro F1:    {macro_f1:.4f}\n")
        f.write(f"  Weighted F1: {weighted_f1:.4f}\n")
    
    print(f"\nSummary:")
    print(f"  Accuracy:    {accuracy:.4f}")
    print(f"  Macro F1:    {macro_f1:.4f}")
    print(f"  Weighted F1: {weighted_f1:.4f}")
    
    # ========== 绘图 ==========
    
    # 1. 混淆矩阵
    cm = confusion_matrix(yte, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        cm, annot=True, fmt="d", cmap="Blues",
        xticklabels=le_emo.classes_, yticklabels=le_emo.classes_,
        annot_kws={"size": 12}
    )
    plt.title("Confusion Matrix — Conformer V3.1", fontsize=16)
    plt.xlabel("Predicted", fontsize=14)
    plt.ylabel("True", fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, "confusion_matrix.png"), dpi=150)
    plt.close()
    
    # 2. 归一化混淆矩阵
    cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        cm_norm, annot=True, fmt=".2%", cmap="Blues",
        xticklabels=le_emo.classes_, yticklabels=le_emo.classes_,
        annot_kws={"size": 11}
    )
    plt.title("Normalized Confusion Matrix — Conformer V3.1", fontsize=16)
    plt.xlabel("Predicted", fontsize=14)
    plt.ylabel("True", fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, "confusion_matrix_normalized.png"), dpi=150)
    plt.close()
    
    # 3. 各类别F1柱状图
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD']
    plt.figure(figsize=(12, 6))
    bars = plt.bar(le_emo.classes_, class_f1, color=colors[:len(le_emo.classes_)])
    plt.axhline(y=macro_f1, color='red', linestyle='--', linewidth=2, 
                label=f'Macro F1 = {macro_f1:.4f}')
    plt.xlabel('Emotion Class', fontsize=14)
    plt.ylabel('F1 Score', fontsize=14)
    plt.title('Per-Class F1 Score — Conformer V3.1', fontsize=16)
    plt.legend(fontsize=12)
    plt.ylim(0, 1)
    for bar, f1 in zip(bars, class_f1):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                 f'{f1:.3f}', ha='center', fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, "class_f1_scores.png"), dpi=150)
    plt.close()
    
    return {
        "accuracy": accuracy,
        "macro_f1": macro_f1,
        "weighted_f1": weighted_f1,
        "class_f1": dict(zip(le_emo.classes_, class_f1))
    }


# =====================
# 15) 绘制训练曲线
# =====================
def plot_training_history(history, plot_dir=PLOT_DIR):
    """绘制并保存训练曲线"""
    
    # 综合图（子图）
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Loss
    axes[0, 0].plot(history.history['loss'], label='Train', linewidth=2)
    axes[0, 0].plot(history.history['val_loss'], label='Val', linewidth=2)
    axes[0, 0].set_title('Loss', fontsize=12)
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Accuracy
    axes[0, 1].plot(history.history['accuracy'], label='Train', linewidth=2)
    axes[0, 1].plot(history.history['val_accuracy'], label='Val', linewidth=2)
    axes[0, 1].set_title('Accuracy', fontsize=12)
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Macro F1
    if 'val_macro_f1' in history.history:
        axes[1, 0].plot(history.history['val_macro_f1'], label='Val Macro F1', 
                        linewidth=2, color='green')
        axes[1, 0].set_title('Validation Macro F1', fontsize=12)
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
    
    # Learning Rate
    if 'lr' in history.history:
        axes[1, 1].plot(history.history['lr'], linewidth=2, color='purple')
        axes[1, 1].set_title('Learning Rate', fontsize=12)
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.suptitle('Training Overview — Conformer V3.1', fontsize=16)
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, "training_overview.png"), dpi=150)
    plt.close()
    print(f"  ✓ Saved: training_overview.png")


# =====================
# 16) 模型对比函数
# =====================
def compare_with_previous(current_results, plot_dir=PLOT_DIR):
    """与之前模型对比"""
    
    previous_results = {
        "CNN": {"macro_f1": 0.4848},
        "Conformer_V1": {"macro_f1": 0.6738},
        "Conformer_V3": {"macro_f1": 0.6505},
        "Conformer_V3.1": {"macro_f1": current_results["macro_f1"]}
    }
    
    models = list(previous_results.keys())
    macro_f1s = [previous_results[m]["macro_f1"] for m in models]
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(models, macro_f1s, color='#3498db', width=0.5)
    
    plt.xlabel('Model', fontsize=14)
    plt.ylabel('Macro F1 Score', fontsize=14)
    plt.title('Model Comparison', fontsize=16)
    plt.ylim(0, 1)
    
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, height, f'{height:.4f}', 
                 ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, "model_comparison.png"), dpi=150)
    plt.close()
    print(f"  ✓ Saved: model_comparison.png")


# =====================
# 17) 主程序
# =====================
if __name__ == "__main__":
    
    print("\n" + "=" * 60)
    print("Speech Emotion Recognition - Conformer V3.1 (Fine-tuned)")
    print("Dataset: CREMA-D")
    print(f"Experiment Directory: {EXP_DIR}")
    print("=" * 60)
    
    # 训练模型
    model, history, best_f1 = train_conformer_v3_1(
        Xtr, ytr, Xte, yte, le_emo,
        epochs=100,
        batch_size=64
    )
    
    # 绘制训练曲线
    print("\nPlotting training curves...")
    plot_training_history(history)
    
    # 评估模型
    print("\nEvaluating model...")
    results = evaluate_model(model, Xte, yte, le_emo)
    
    # 与之前模型对比
    print("\nComparing with previous models...")
    compare_with_previous(results)
    
    # 保存最终结果
    print("\n" + "=" * 60)
    print("Final Results")
    print("=" * 60)
    print(f"Accuracy:    {results['accuracy']:.4f}")
    print(f"Macro F1:    {results['macro_f1']:.4f}")
    print(f"Weighted F1: {results['weighted_f1']:.4f}")
    
    print("\n" + "=" * 60)
    print(f"✓ All results saved to: {EXP_DIR}")
    print("=" * 60)

2025-12-04 03:25:48.567941: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-04 03:25:48.685852: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-04 03:25:49.726944: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version: 2.13.1
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✓ Experiment directory created: VED/Output/conformer_v3_1_notebook_20251204_032550
Total samples: 7442
Emotion distribution:
emotion
ANG    1271
DIS    1271
FEA    1271
HAP    1271
SAD    1271
NEU    1087
Name: count, dtype: int64
Feature shape: (300, 64, 1)
Extracting features...
  Processing 0/7442...
  Processing 500/7442...
  Processing 1000/7442...
  Processing 1500/7442...
  Processing 2000/7442...
  Processing 2500/7442...
  Processing 3000/7442...
  Processing 3500/7442...
  Processing 4000/7442...
  Processing 4500/7442...
  Processing 5000/7442...
  Processing 5500/7442...
  Processing 6000/7442...
  Processing 6500/7442...
  Processing 7000/7442...
Feature matrix shape: (7442, 300, 64, 1)
Classes: ['ANG' 'DIS' 'FEA' 'HAP' 'NEU' 'SAD']
Number of classes: 6
Train: (5890, 300, 64, 1), Test: (1552, 300, 64, 1)
Normalized - Train mean: -0.0000, std: 1.0000

Speech Emotion 

2025-12-04 03:26:48.022411: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31132 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:65:04.0, compute capability: 7.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 300, 64, 1)]         0         []                            
                                                                                                  
 gaussian_noise (GaussianNo  (None, 300, 64, 1)           0         ['input_1[0][0]']             
 ise)                                                                                             
                                                                                                  
 spec_augment (SpecAugment)  (None, 300, 64, 1)           0         ['gaussian_noise[0][0]']      
                                                                                                  
 conv2d (Conv2D)             (None, 300, 64, 32)          320       ['spec_augment[0][0]']    

2025-12-04 03:26:59.894706: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/spatial_dropout2d/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2025-12-04 03:27:01.467529: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2025-12-04 03:27:02.114149: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55b304962ef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-04 03:27:02.114191: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2025-12-04 03:27:02.123902: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-04 03:27:02.308019: I ./tensorflow/compiler/jit/device_compiler.h:186] Co

...

93/93 [==============================] - 12s 125ms/step - loss: 0.3476 - accuracy: 0.9000 - val_loss: 0.7319 - val_accuracy: 0.6643 - val_macro_f1: 0.6606 - lr: 1.3916e-05
Epoch 94/100
93/93 [==============================] - ETA: 0s - loss: 0.3693 - accuracy: 0.8910 — val_macro_f1: 0.6587

Epoch 94: val_macro_f1 did not improve from 0.67073


/root/miniconda3/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


93/93 [==============================] - 12s 125ms/step - loss: 0.3693 - accuracy: 0.8910 - val_loss: 0.7252 - val_accuracy: 0.6624 - val_macro_f1: 0.6587 - lr: 1.0669e-05
Epoch 95/100
93/93 [==============================] - ETA: 0s - loss: 0.3233 - accuracy: 0.9080 — val_macro_f1: 0.6548

Epoch 95: val_macro_f1 did not improve from 0.67073


/root/miniconda3/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Restoring model weights from the end of the best epoch: 70.
93/93 [==============================] - 12s 125ms/step - loss: 0.3233 - accuracy: 0.9080 - val_loss: 0.7316 - val_accuracy: 0.6585 - val_macro_f1: 0.6548 - lr: 7.8480e-06
Epoch 95: early stopping

Plotting training curves...
  ✓ Saved: training_overview.png

Evaluating model...

Classification Report - Conformer V3.1
              precision    recall  f1-score   support

         ANG     0.7424    0.8264    0.7821       265
         DIS     0.6681    0.5698    0.6151       265
         FEA     0.5953    0.6717    0.6312       265
         HAP     0.6725    0.5811    0.6235       265
         NEU     0.7273    0.8106    0.7667       227
         SAD     0.6240    0.5887    0.6058       265

    accuracy                         0.6714      1552
   macro avg     0.6716    0.6747    0.6707      1552
weighted avg     0.6702    0.6714    0.6684      1552


Summary:
  Accuracy:    0.6714
  Macro F1:    0.6707
  Weighted F1: 0.6684

